Idea is to read in the .tsv file of interproscan and pull out the following when having run interproscan with the following option.
This was run like:
$INTPRO/interproscan.sh -i ../../../v91_cns_gcoords_curs_ph_ctg_p_ctg.evm.all.protein.fa -iprlookup -goterms -pa
and produced following file
v91_cns_gcoords_curs_ph_ctg_p_ctg.evm.all.protein.intpro.fa.tsv


In [1]:
import pandas as pd
import os
import re
from Bio import SeqIO
import pysam
from Bio.SeqRecord import SeqRecord
from pybedtools import BedTool
import numpy as np
import pybedtools
import time
import matplotlib.pyplot as plt
import sys
import subprocess
import shutil

In [2]:
BASE_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/funnotate/Pst_79a/intpro'
INTERPRO_TSV_FILE = 'v91_cns_gcoords_cursf_ph_ctg_a_ctg.evm.all.protein.intpro.fa.tsv'
BASE_AA_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12'
BASE_A_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
ANNOTATION_FILE_BASE_PATH ='/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/funnotate/Pst_79a/fun-out/annotate_misc'
OUT_PATH = os.path.join(BASE_AA_PATH, 'protein_annotation')
if not os.path.exists(OUT_PATH):
    os.mkdir(OUT_PATH)

In [3]:
p_genome = 'Pst_104E_v12_h_ctg'
OUT_PATH_P = os.path.join(OUT_PATH, p_genome)
if not os.path.exists(OUT_PATH_P):
    os.mkdir(OUT_PATH_P)
#that is the path for all the proteins without removing the high coverage contigs and the proteins w/ 
#similarities to TE proteins
OUT_PATH_P_ALL = os.path.join(OUT_PATH_P, 'ALL')
if not os.path.exists(OUT_PATH_P_ALL):
    os.mkdir(OUT_PATH_P_ALL)

In [4]:
#pull out all proteins that are in the final assembly
p_protein_list = []
protein_fa_file = [x for x in os.listdir(BASE_A_PATH) if p_genome in x and x.endswith('anno.protein.fa')][0]
for protein in SeqIO.parse(os.path.join(BASE_A_PATH, protein_fa_file) , 'fasta'):
    p_protein_list.append(protein.id)

In [5]:
interpro_header = ['Protein_ID' , 'MD5', 'Length', 'DB', 'DB_accession', 'DB_description', 'Start_position', 'Stop_position', \
                  'e-value', 'Match Status', 'date', 'InterPro_ID', 'InterPro_description','GO_terms', 'Pathway_IDs' ]

In [6]:
interpro_df = pd.read_csv(os.path.join(BASE_FOLDER, INTERPRO_TSV_FILE), sep ='\t', header=None, names=interpro_header)

In [17]:
#rename protein names depending on the genome h or p
def protein_id_conversion(old_protein_df):
    '''Function that converts a old protein ID from v91 version to Pst_104E_v12 version. 
    E.g. 	evm.model.000004F_quiver.189 to evm.model.pcontig_004.189.
    Input is a pandas series and output is a pandas series of same length.
    '''
    if p_genome.endswith('p_ctg'):
        rename_df = old_protein_df.Protein_ID.str.extract(r'000([0-9]*)F_quiver.([0-9]*)')
        rename_df['Updated_Protein_ID'] = 'evm.model.pcontig_' + rename_df[0] + '.' + rename_df[1]
    elif p_genome.endswith('h_ctg'):
        rename_df = old_protein_df.Protein_ID.str.extract(r'000([0-9]*)F_q?u?i?v?e?r?_?0?0?0?([0-9]*)F?_quiver.([0-9]*)')
        
        rename_df['Updated_Protein_ID'] = 'evm.model.hcontig_' + rename_df[0] + '_' + rename_df[1] +'.' + rename_df[2]
    return rename_df['Updated_Protein_ID']

In [18]:
interpro_df['Updated_Protein_ID'] = protein_id_conversion(interpro_df)

/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [19]:
interpro_df.head()

,Protein_ID,MD5,Length,DB,DB_accession,DB_description,Start_position,Stop_position,e-value,Match Status,date,InterPro_ID,InterPro_description,GO_terms,Pathway_IDs,Updated_Protein_ID,Dbxref
0,evm.model.000014F_quiver_000117F_quiver.28,f7d31a00c92d480a23f48a3138c52642,308,TMHMM,TMhelix,Region of a membrane-bound protein predicted t...,235,257,-,T,19-11-2016,0,NaN,0,0,evm.model.hcontig_014_117.28,Dbxref
1,evm.model.000014F_quiver_000117F_quiver.28,f7d31a00c92d480a23f48a3138c52642,308,MobiDBLite,mobidb-lite,consensus disorder prediction,285,308,-,T,19-11-2016,0,NaN,0,0,evm.model.hcontig_014_117.28,Dbxref
2,evm.model.000003F_002_quiver.160,9c25d3497e14151d2c486b5d57addc34,181,PANTHER,PTHR33069:SF3,NaN,20,175,3.5E-38,T,19-11-2016,0,NaN,0,0,evm.model.hcontig_003_2.160,Dbxref
3,evm.model.000003F_002_quiver.160,9c25d3497e14151d2c486b5d57addc34,181,PANTHER,PTHR33069,NaN,20,175,3.5E-38,T,19-11-2016,0,NaN,0,0,evm.model.hcontig_003_2.160,Dbxref
4,evm.model.000055F_012_quiver.53,924e1702ee24a994dc950eb4560364d8,151,CDD,cd04413,NDPk_I,5,134,2.89677E-86,T,19-11-2016,0,NaN,0,0,evm.model.hcontig_055_12.53,Dbxref


In [20]:
interpro_df.GO_terms.fillna(0, inplace = True)
interpro_df.Pathway_IDs.fillna(0, inplace =True)
interpro_df.InterPro_ID.fillna(0,inplace =True)

In [21]:
interpro_by_protein = interpro_df.groupby('Updated_Protein_ID')

In [23]:
DBs = interpro_df.DB.unique()
interpro_df['Dbxref'] = 'Dbxref'
for db in DBs:
    interpro_df[(interpro_df.DB == db) & (interpro_df.Updated_Protein_ID.isin(p_protein_list))].loc[:,['Updated_Protein_ID', 'DB']]\
    .to_csv(os.path.join(OUT_PATH_P, db+'_terms.tab'), sep='\t', header =None, index = None)
    interpro_df[(interpro_df.DB == db) & (interpro_df.Updated_Protein_ID.isin(p_protein_list))].loc[:,['Updated_Protein_ID', 'Dbxref','DB']]\
    .to_csv(os.path.join(OUT_PATH_P, 'annotations.' +db+'.txt'), sep='\t', header =None, index = None)
    interpro_df[(interpro_df.DB == db) ].loc[:,['Updated_Protein_ID', 'DB']]\
    .to_csv(os.path.join(OUT_PATH_P_ALL, db+'_terms_all.tab'), sep='\t', header =None, index = None)
    interpro_df[(interpro_df.DB == db) ].loc[:,['Updated_Protein_ID', 'Dbxref','DB']]\
    .to_csv(os.path.join(OUT_PATH_P_ALL, 'annotations.' +db+'_all.txt'), sep='\t', header =None, index = None)
    

In [26]:
#write out the interpro domains
interpro_df['InterPro'] = 'InterPro'
interpro_df[(interpro_df.InterPro_ID != 0) & (interpro_df.Updated_Protein_ID.isin(p_protein_list))].loc[:,['Updated_Protein_ID', 'InterPro_ID']].to_csv(os.path.join(OUT_PATH_P, 'iprscan_terms.tab'), sep='\t', header =None, index = None)
interpro_df[interpro_df.InterPro_ID != 0].loc[:,['Updated_Protein_ID', 'InterPro_ID']].to_csv(os.path.join(OUT_PATH_P_ALL, 'iprscan_terms_all.tab'), sep='\t', header =None, index = None)
#write out annotations
interpro_df[(interpro_df.InterPro_ID != 0) & (interpro_df.Updated_Protein_ID.isin(p_protein_list))]\
.loc[:,['Updated_Protein_ID', 'InterPro','InterPro_ID']].to_csv(os.path.join(OUT_PATH_P, 'annotations.iprscan.txt'), sep='\t', header =None, index = None)
#write out annotations for all proteins
interpro_df[interpro_df.InterPro_ID != 0].loc[:,['Updated_Protein_ID', 'InterPro','InterPro_ID']].to_csv(os.path.join(OUT_PATH_P_ALL, 'annotations.iprscan_all.tab'), sep='\t', header =None, index = None)

In [27]:
#process the pathway files
interpro_by_protein_KEGG = interpro_df[interpro_df.Pathway_IDs.str.contains('KEGG' or 'MetaCyc' or 'Reactome').fillna(False)]

interpro_by_protein_KEGG = interpro_by_protein_KEGG.groupby('Updated_Protein_ID')

#pull out all the KEGG terms and write them out as annotation files 
interpro_by_protein_KEGG = interpro_by_protein.Pathway_IDs.apply(set)

#remove everything without KEGG term attached
interpro_by_protein_KEGG = interpro_by_protein_KEGG[~(interpro_by_protein_KEGG == {0})]

interpro_by_protein_KEGG_dict = dict(zip(interpro_by_protein_KEGG.index, interpro_by_protein_KEGG))

ALL_KEGG_LIST = []
ALL_PROTEIN_INDEX_LIST = []
for key in list(interpro_by_protein_KEGG_dict.keys()):
    _tmp_list = list(interpro_by_protein_KEGG_dict[key])
    #remove 0 
    _tmp_list = [x for x in _tmp_list if x != 0]
    new_value = []
    for x in _tmp_list: 
        if '|' in x:
            _list = x.split('|')
            for y in _list:
                new_value.append(y)
        else:
            new_value.append(x)
    new_value = list(set(new_value))
    new_index = [key]*len(new_value)
    ALL_KEGG_LIST += new_value
    ALL_PROTEIN_INDEX_LIST += new_index


KEGG_df = pd.DataFrame([ALL_PROTEIN_INDEX_LIST, ALL_KEGG_LIST]).T
KEGG_df.rename(columns={0:'Updated_Protein_ID', 1:'DB_ID'}, inplace=True)

KEGG_df.to_csv(os.path.join(OUT_PATH_P_ALL, 'Pathway_terms_ipr_all.tab') , sep = '\t', header =None, index=None)
KEGG_df[KEGG_df.Updated_Protein_ID.isin(p_protein_list)].to_csv(os.path.join(OUT_PATH_P, 'Pathway_terms_ipr.tab') , sep = '\t', header =None, index=None)
#write out annotations
KEGG_df['Transfer_ID'] = 'note'
KEGG_df.loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P_ALL, 'annotations.Pathway_all.txt') , sep = '\t', header =None, index=None)
KEGG_df[KEGG_df.Updated_Protein_ID.isin(p_protein_list)].loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P, 'annotations.Pathway.txt') , sep = '\t', header =None, index=None)

In [28]:
#pull out all the GO terms and write them out as annotation files 
interpro_by_protein_GO = interpro_by_protein.GO_terms.apply(set)

#remove everything without GO term attached
interpro_by_protein_GO = interpro_by_protein_GO[~(interpro_by_protein_GO == {0})]

interpro_by_protein_GO_dict = dict(zip(interpro_by_protein_GO.index, interpro_by_protein_GO))

ALL_GO_LIST = []
ALL_PROTEIN_INDEX_LIST = []
for key in list(interpro_by_protein_GO_dict.keys()):
    _tmp_list = list(interpro_by_protein_GO_dict[key])
    #remove 0 
    _tmp_list = [x for x in _tmp_list if x != 0]
    new_value = []
    for x in _tmp_list: 
        if '|' in x:
            _list = x.split('|')
            for y in _list:
                new_value.append(y)
        else:
            new_value.append(x)
    new_value = list(set(new_value))
    new_index = [key]*len(new_value)
    ALL_GO_LIST += new_value
    ALL_PROTEIN_INDEX_LIST += new_index


GO_df = pd.DataFrame([ALL_PROTEIN_INDEX_LIST, ALL_GO_LIST]).T
GO_df.rename(columns={0:'Updated_Protein_ID', 1:'DB_ID'}, inplace=True)

GO_df.to_csv(os.path.join(OUT_PATH_P_ALL, 'GO_terms_ipr_all.tab') , sep = '\t', header =None, index=None)
GO_df[GO_df.Updated_Protein_ID.isin(p_protein_list)].to_csv(os.path.join(OUT_PATH_P, 'GO_terms_ipr.tab') , sep = '\t', header =None, index=None)
#write out annotations
GO_df['Transfer_ID'] = 'note'
GO_df.loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P_ALL, 'annotations.GO_all.txt') , sep = '\t', header =None, index=None)
GO_df[GO_df.Updated_Protein_ID.isin(p_protein_list)].loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P, 'annotations.GO.txt') , sep = '\t', header =None, index=None)

In [29]:
#filter out other annotations files including busco, dbCAN, merops, swissprot
annotation_files = [os.path.join(ANNOTATION_FILE_BASE_PATH, x) for x in os.listdir(ANNOTATION_FILE_BASE_PATH) \
                    if x.startswith('annotations') and (x.endswith('busco.txt') or x.endswith('dbCAN.txt') or x.endswith('merops.txt') or x.endswith('swissprot.txt'))]

In [30]:
for anno in annotation_files:
    _tmp_df = pd.read_csv(anno, header=None, sep='\t', names=['Protein_ID','Transfer_ID', 'DB_ID'])
    anno_file_name = anno.split('/')[-1]
    anno_midfix = anno_file_name.split('.')[1]
    _tmp_df['Updated_Protein_ID'] = protein_id_conversion(_tmp_df)
    #write out filtered down tab file
    _tmp_df[(_tmp_df.Updated_Protein_ID.isin(p_protein_list))].loc[:,['Updated_Protein_ID', 'DB_ID']].to_csv(os.path.join(OUT_PATH_P, anno_midfix+'_terms.tab'), sep = '\t', header =None, index=None)
    #write out the filtered annotation file
    _tmp_df[(_tmp_df.Updated_Protein_ID.isin(p_protein_list))].loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].\
    to_csv(os.path.join(OUT_PATH_P, anno_file_name ), sep = '\t', header =None, index=None)
     #write out not down tab file
    _tmp_df.loc[:,['Updated_Protein_ID', 'DB_ID']].to_csv(os.path.join(OUT_PATH_P_ALL, anno_midfix+'_terms_all.tab' ), sep = '\t', header =None, index=None)
    #write out the filtered annotation file
    _tmp_df.loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P_ALL, anno_file_name.replace('.txt', '_all.txt') ), sep = '\t', header =None, index=None)

/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
